In [2]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='localhost', port=int(3306), user='root',passwd='1234', db='sakila')
cursor = conn.cursor(pymysql.cursors.DictCursor)

### Subqueries
* a query contained within another SQL statment.
* It is always enclosed within parentheses


In [3]:
sen = '''

SELECT customer_id, first_name, last_name
FROM customer
WHERE customer_id =(SELECT MAX(customer_id) From customer)

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,customer_id,first_name,last_name
0,599,AUSTIN,CINTRON


### Subquery Types
* single row/column
* single row/multicolumn
* multiple column
* noncorrelated subqueries
* correlated subqueries


In [4]:
sen = '''

SELECT city_id, city
FROM city
WHERE country_id <>
    (SELECT country_id FROM country WHERE country ='india')

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,city_id,city
0,1,A Corua (La Corua)
1,2,Abha
2,3,Abu Dhabi
3,4,Acua
4,5,Adana
...,...,...
535,596,Zaria
536,597,Zeleznogorsk
537,598,Zhezqazghan
538,599,Zhoushan


### Single row/ column이 필요한 이유

WHERE country_id 는 하나의 값만 받을 수 있는데 반환된 값은 하나의 set이기 때문에 오류가 발생함

In [5]:
sen = '''

SELECT city_id, city
FROM city
WHERE country_id <>
    (SELECT country_id FROM country WHERE country !='india')

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

OperationalError: (1242, 'Subquery returns more than 1 row')

### multi-row, single-column
하나의 set을 활용하는 query는 네 종류가 있다.

1. in and not in operators

In [ ]:
sen = '''

SELECT country_id
FROM country
WHERE country IN ('Canada','Mexico');

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,country_id
0,20
1,60


In [ ]:
sen = '''

SELECT city_id, city
FROM city
WHERE country_id IN
    (
        SELECT country_id 
        FROM country
        WHERE country IN('Canada','Mexico')
    );

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,city_id,city
0,179,Gatineau
1,196,Halifax
2,300,Lethbridge
3,313,London
4,383,Oshawa
5,430,Richmond Hill
6,565,Vancouver
7,4,Acua
8,19,Allende
9,40,Atlixco


In [ ]:
sen = '''

SELECT city_id, city
FROM city
WHERE country_id IN
    (
        SELECT country_id 
        FROM country
        WHERE country NOT IN('Canada','Mexico')
    );

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,city_id,city
0,251,Kabul
1,59,Batna
2,63,Bchar
3,483,Skikda
4,516,Tafuna
...,...,...
558,455,Sanaa
559,518,Taizz
560,280,Kragujevac
561,368,Novi Sad


2. The all operator


   `ALL operator` allows you to make comparsions between a single value and every value in a set

In [ ]:
sen = '''

SELECT first_name, last_name
FROM customer
WHERE customer_id <> ALL
    (
        SELECT customer_id 
        FROM payment
        WHERE amount =0
    );

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,first_name,last_name
0,MARY,SMITH
1,PATRICIA,JOHNSON
2,LINDA,WILLIAMS
3,BARBARA,JONES
4,ELIZABETH,BROWN
...,...,...
571,TERRENCE,GUNDERSON
572,ENRIQUE,FORSYTHE
573,FREDDIE,DUGGAN
574,WADE,DELVALLE


### NOT IN 또는 <> ALL 사용 시 주의할 점

Null VALUE가 포함되면 return으로 아무것도 반환하지 않는다.

In [ ]:
sen = '''

SELECT city_id, city
FROM city
WHERE country_id IN (123, NULL)

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

""


3. any operator

ALL과 마찬가지로 ANY도 SET의 element를 하나씩 꺼내서 비교한다.


ANY 설명이 좀 빈약하다. 나중에 더 찾아봐야겠다.

### Multicolumn subqueries

In [ ]:
sen = '''

SELECT fa.actor_id, fa.film_id
FROM film_actor fa
WHERE fa.actor_id IN
    ( SELECT actor_id FROM actor WHERE last_name = 'MONROE')
    AND fa.film_id IN 
    ( SELECT film_id FROM film WHERE rating = 'PG');

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id,film_id
0,120,63
1,120,144
2,120,414
3,120,590
4,120,715
5,120,894
6,178,164
7,178,194
8,178,273
9,178,311


### 위 statement와 아래 statment의 결과는 같다.

Multi column을 어떻게 대처하는지 배울 수 있다.

In [ ]:
sen = '''

SELECT fa.actor_id, fa.film_id
FROM film_actor fa
WHERE (fa.actor_id, film_id) IN
    (
        SELECT a.actor_id, f.film_id
        FROM actor a
            CROSS JOIN film f
        WHERE a.last_name ='MONROE'
        AND f.rating = 'PG'
    );

'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id,film_id
0,120,63
1,120,144
2,120,414
3,120,590
4,120,715
5,120,894
6,178,164
7,178,194
8,178,273
9,178,311


### Correlated subqueries

unrelated queries는 최종 테이블을 만들기 전에 실행됨. 반면 correlated queries는 table이 만들어진 후 검색 용도로서 활용됨

correlated subqueries는 row 하나씩 불러온다고 한다. 이 말은 데이터가 클 경우에 오랜 시간이 걸린다는 의미

In [ ]:
sen = '''

SELECT c.first_name, c.last_name
FROM customer c
WHERE 20 = (SELECT count(*) FROM rental r 
            WHERE r.customer_id = c.customer_id);


'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,first_name,last_name
0,LAUREN,HUDSON
1,JEANETTE,GREENE
2,TARA,RYAN
3,WILMA,RICHARDS
4,JO,FOWLER
5,KAY,CALDWELL
6,DANIEL,CABRAL
7,ANTHONY,SCHWAB
8,TERRY,GRISSOM
9,LUIS,YANEZ


In [ ]:
sen = '''

SELECT c.first_name, c.last_name
FROM customer c
WHERE 
    ( SELECT sum(p.amount) FROM payment p
      WHERE p.customer_id = c.customer_id )
      BETWEEN 180 AND 240;


'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,first_name,last_name
0,RHONDA,KENNEDY
1,CLARA,SHAW
2,ELEANOR,HUNT
3,MARION,SNYDER
4,TOMMY,COLLAZO
5,KARL,SEAL


### The exsists operator

correlated subqueries를 효율적으로 활용하는 operator라고 한다. 존재 여부만 체크하는데 사용된다고 한다. 

> quantity를 따져야 하는 경우가 아니면 더 빠른 결과를 발생시키는가 봄

In [ ]:
sen = '''

SELECT c.first_name, c.last_name
FROM customer c
WHERE EXISTS
    (SELECT 1 FROM rental r
     WHERE r.customer_id = c.customer_id
     AND date(r.rental_date) < '2005-05-25'
     );


'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,first_name,last_name
0,CHARLOTTE,HUNTER
1,DELORES,HANSEN
2,MINNIE,ROMERO
3,CASSANDRA,WALTERS
4,ANDREW,PURDY
5,MANUEL,MURRELL
6,TOMMY,COLLAZO
7,NELSON,CHRISTENSON


### When to Use Subqueries

1. Subqueries as Data Sources
   
   테이블에 없는 데이터를 가공해서 만든 다음 새로운 table에 넣는다.

In [10]:
sen = '''

SELECT c.first_name, c.last_name, pymnt.num_rentals, pymnt.tot_payments
FROM customer c
    INNER JOIN 
    (SELECT customer_id, count(*) num_rentals, sum(amount) tot_payments 
    FROM payment 
    GROUP BY customer_id) pymnt
    ON c.customer_id = pymnt.customer_id;
'''
cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,first_name,last_name,num_rentals,tot_payments
0,MARY,SMITH,32,118.68
1,PATRICIA,JOHNSON,27,128.73
2,LINDA,WILLIAMS,26,135.74
3,BARBARA,JONES,22,81.78
4,ELIZABETH,BROWN,38,144.62
...,...,...,...,...
594,TERRENCE,GUNDERSON,30,117.70
595,ENRIQUE,FORSYTHE,28,96.72
596,FREDDIE,DUGGAN,25,99.75
597,WADE,DELVALLE,22,83.78


2. Data Fabrication